<a href="https://colab.research.google.com/github/dustiny5/DS-Unit-2-Sprint-3-Classification-Validation/blob/master/module1-logistic-regression/LS_DS_231_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - Logistic Regression

Logistic regression is the baseline for classification models, as well as a handy way to predict probabilities (since those too live in the unit interval). While relatively simple, it is also the foundation for more sophisticated classification techniques such as neural networks (many of which can effectively be thought of as networks of logistic models).

## Lecture - Where Linear goes Wrong
### Return of the Titanic 🚢

You've likely already explored the rich dataset that is the Titanic - let's use regression and try to predict survival with it. The data is [available from Kaggle](https://www.kaggle.com/c/titanic/data), so we'll also play a bit with [the Kaggle API](https://github.com/Kaggle/kaggle-api).

### Get data, option 1: Kaggle API

#### Sign up for Kaggle and get an API token
1. [Sign up for a Kaggle account](https://www.kaggle.com/), if you don’t already have one. 
2. [Follow these instructions](https://github.com/Kaggle/kaggle-api#api-credentials) to create a Kaggle “API Token” and download your `kaggle.json` file. If you are using Anaconda, put the file in the directory specified in the instructions.

_This will enable you to download data directly from Kaggle. If you run into problems, don’t worry — I’ll give you an easy alternative way to download today’s data, so you can still follow along with the lecture hands-on. And then we’ll help you through the Kaggle process after the lecture._

#### Put `kaggle.json` in the correct location

- ***If you're using Anaconda,*** put the file in the directory specified in the [instructions](https://github.com/Kaggle/kaggle-api#api-credentials).

- ***If you're using Google Colab,*** upload the file to your Google Drive, and run this cell:

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/


#### Install the Kaggle API package and use it to get the data

You also have to join the Titanic competition to have access to the data

In [2]:
!pip install kaggle

In [3]:
!kaggle competitions download -c titanic

train.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)
gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)


### Get data, option 2: Download from the competition page
1. [Sign up for a Kaggle account](https://www.kaggle.com/), if you don’t already have one. 
2. [Go to the Titanic competition page](https://www.kaggle.com/c/titanic) to download the [data](https://www.kaggle.com/c/titanic/data).

### Get data, option 3: Use Seaborn

```
import seaborn as sns
train = sns.load_dataset('titanic')
```

But Seaborn's version of the Titanic dataset is not identical to Kaggle's version, as we'll see during this lesson!

### Read data

In [4]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.shape,test.shape

((891, 12), (418, 11))

In [5]:
train.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
269,270,1,1,"Bissette, Miss. Amelia",female,35.0,0,0,PC 17760,135.6333,C99,S
711,712,0,1,"Klaber, Mr. Herman",male,NaN,0,0,113028,26.5500,C124,S
849,850,1,1,"Goldenberg, Mrs. Samuel L (Edwiga Grabowska)",female,NaN,1,0,17453,89.1042,C92,C
845,846,0,3,"Abbing, Mr. Anthony",male,42.0,0,0,C.A. 5547,7.5500,NaN,S
566,567,0,3,"Stoytcheff, Mr. Ilia",male,19.0,0,0,349205,7.8958,NaN,S


In [6]:
test.sample(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
190,1082,2,"Angle, Mr. William A",male,34.0,1,0,226875,26.0000,NaN,S
272,1164,1,"Clark, Mrs. Walter Miller (Virginia McDowell)",female,26.0,1,0,13508,136.7792,C89,C
402,1294,1,"Gibson, Miss. Dorothy Winifred",female,22.0,0,1,112378,59.4000,NaN,C
124,1016,3,"Kennedy, Mr. John",male,NaN,0,0,368783,7.7500,NaN,Q
379,1271,3,"Asplund, Master. Carl Edgar",male,5.0,4,2,347077,31.3875,NaN,S


In [7]:
target = 'Survived'
train[target].value_counts(normalize=True)

0    0.616162
1    0.383838
Name: Survived, dtype: float64

In [8]:
train.describe(include='number')

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [9]:
train.describe(exclude='number')

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Brown, Mrs. James Joseph (Margaret Tobin)",male,347082,G6,S
freq,1,577,7,4,644


### How would we try to do this with linear regression?

https://scikit-learn.org/stable/modules/impute.html

In [10]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

features = ['Pclass', 'Age', 'Fare']
target = 'Survived'

X_train = train[features] 
y_train = train[target]
X_test = test[features]

imputer = SimpleImputer()#Default fit NaN with the mean values
X_train_imputed = imputer.fit_transform(X_train)#Fill NaN values 
X_test_imputed = imputer.fit_transform(X_test)#Fill NaN values from train data

lin_reg = LinearRegression()
lin_reg.fit(X_train_imputed, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [11]:
X_train.shape, X_train_imputed.shape, X_test.shape, X_test_imputed.shape

((891, 3), (891, 3), (418, 3), (418, 3))

In [12]:
X_train['Age'].mean()

29.69911764705882

In [13]:
X_test.tail()

,Pclass,Age,Fare
413,3,NaN,8.0500
414,1,39.0,108.9000
415,3,38.5,7.2500
416,3,NaN,8.0500
417,3,NaN,22.3583


In [14]:
X_test_imputed[-5:]

array([[  3.        ,  30.27259036,   8.05      ],
       [  1.        ,  39.        , 108.9       ],
       [  3.        ,  38.5       ,   7.25      ],
       [  3.        ,  30.27259036,   8.05      ],
       [  3.        ,  30.27259036,  22.3583    ]])

In [15]:
import numpy as np

#Rich 5 year old in first class
test_case = np.array([[1, 5, 500]])
lin_reg.predict(test_case)

array([1.19207871])

In [0]:
y_pred = lin_reg.predict(X_test_imputed)

In [17]:
pd.Series(y_pred).describe()

count    418.000000
mean       0.391255
std        0.182398
min        0.014324
25%        0.225541
50%        0.339570
75%        0.516439
max        0.954827
dtype: float64

In [18]:
pd.Series(lin_reg.coef_, X_train.columns)

Pclass   -0.210390
Age      -0.007358
Fare      0.000751
dtype: float64

In [19]:
lin_reg.intercept_

1.063899500003544

### How would we do this with Logistic Regression?

In [20]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='lbfgs')
lr.fit(X_train_imputed, y_train)

print('Prediction for rich 5 year old', lr.predict(test_case))
print('Prediction probabilities for rich 5 year old', lr.predict_proba(test_case))

Prediction for rich 5 year old [1]
Prediction probabilities for rich 5 year old [[0.02778799 0.97221201]]


In [21]:
#lr.predict(X_test_imputed)
threshold = 0.5
(lr.predict_proba(X_test_imputed)[:,1] > threshold).astype(int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,

In [22]:
manual_predictions = (lr.predict_proba(X_test_imputed)[:,1] > threshold).astype(int)
direct_predictions = lr.predict(X_test_imputed)

#Returns True if all are true and false if at least 1 is false
all(manual_predictions == direct_predictions)

True

### How accurate is the Logistic Regression?

In [23]:
#Accuracy Score for train
lr.score(X_train_imputed, y_train)

0.7025813692480359

In [24]:
#Unable to test because it is in Kaggle
lr.score(X_test_imputed, y_train)

ValueError: ignored

In [0]:
#Cross Validation
from sklearn.model_selection import cross_val_score
score = cross_val_score(lr, X_train_imputed, y_train, cv=10)

In [26]:
score.min(), score.mean(), score.max()

(0.6222222222222222, 0.7016408466689366, 0.7640449438202247)

In [0]:
y_pred = lr.predict(X_train_imputed)

In [28]:
y_pred[:5]

array([0, 1, 0, 1, 0])

In [29]:
y_train[:5].values

array([0, 1, 1, 1, 0])

In [30]:
#Compare y_train and y_pred
correct_pred = 4
total_pred = 5
accuracy = correct_pred / total_pred
accuracy

0.8

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train[:5], y_pred[:5])

0.8

In [32]:
y_train.value_counts(normalize=True)

0    0.616162
1    0.383838
Name: Survived, dtype: float64

### What's the math for the Logistic Regression?

https://en.wikipedia.org/wiki/Logistic_function

https://en.wikipedia.org/wiki/Logistic_regression#Probability_of_passing_an_exam_versus_hours_of_study

In [33]:
lr.coef_, lr.intercept_

(array([[-0.9345267 , -0.03569729,  0.00422069]]), array([2.55763985]))

In [0]:
#Logistic sigmoid "squishing" fxn
#Implemented to work with numpy arrays

def sigmoid(x):
  return 1 / (1 + np.e**(-x))

In [35]:
sigmoid(np.dot(lr.coef_, test_case.T) + lr.intercept_)

array([[0.97221201]])

In [36]:
sigmoid((lr.coef_ @ test_case.T) + lr.intercept_)

array([[0.97221201]])

In [37]:
lr.predict_proba(test_case)

array([[0.02778799, 0.97221201]])

## Feature Engineering

Get the [Category Encoder](http://contrib.scikit-learn.org/categorical-encoding/) library

If you're running on Google Colab:

```
!pip install category_encoders
```

If you're running locally with Anaconda:

```
!conda install -c conda-forge category_encoders
```

In [38]:
!pip install category_encoders

In [0]:
import seaborn as sns
sns_titanic = sns.load_dataset('titanic')

In [40]:
train.shape

(891, 12)

In [41]:
sns_titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [42]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [0]:
def make_features(X):
  X = X.copy()
  X['adult_male'] = (X['Sex'] =='male') & (X['Age'] >= 16)
  X['alone'] = (X['SibSp'] == 0) & (X['Parch'] == 0)
  X['last_name'] = X['Name'].str.split(',').str[0]
  X['title'] = X['Name'].str.split(',').str[1].str.split('.').str[0]
  return X

In [0]:
train = make_features(train)
test = make_features(test)

In [45]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,adult_male,alone,last_name,title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,True,False,Braund,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,False,False,Cumings,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,False,True,Heikkinen,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,False,False,Futrelle,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,True,True,Allen,Mr


In [46]:
train['title'].value_counts()

 Mr              517
 Miss            182
 Mrs             125
 Master           40
 Dr                7
 Rev               6
 Mlle              2
 Major             2
 Col               2
 Lady              1
 Capt              1
 Mme               1
 Jonkheer          1
 Don               1
 Ms                1
 the Countess      1
 Sir               1
Name: title, dtype: int64

## Assignment: real-world classification

We're going to check out a larger dataset - the [FMA Free Music Archive data](https://github.com/mdeff/fma). It has a selection of CSVs with metadata and calculated audio features that you can load and try to use to classify genre of tracks. To get you started:

### Get and unzip the data

#### Google Colab

In [47]:
!wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
!unzip fma_metadata.zip

--2019-05-06 21:15:31--  https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
Resolving os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)... 86.119.28.13, 2001:620:5ca1:2ff::ce53
Connecting to os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)|86.119.28.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358412441 (342M) [application/zip]
Saving to: ‘fma_metadata.zip.1’

fma_metadata.zip.1  100%[===================>] 341.81M  18.4MB/s    in 19s     

2019-05-06 21:15:52 (17.6 MB/s) - ‘fma_metadata.zip.1’ saved [358412441/358412441]

Archive:  fma_metadata.zip
replace fma_metadata/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 bunzipping: fma_metadata/README.txt  
replace fma_metadata/checksums? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 bunzipping: fma_metadata/checksums  
replace fma_metadata/not_found.pickle? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 bunzipping: fma_metadata/not_found.pickle  
replace fma_metadata/raw_genres.csv? [y]es, [n]o, [A]ll, [N]one,

#### Windows
- Download the [zip file](https://os.unil.cloud.switch.ch/fma/fma_metadata.zip)
- You may need to use [7zip](https://www.7-zip.org/download.html) to unzip it


#### Mac
- Download the [zip file](https://os.unil.cloud.switch.ch/fma/fma_metadata.zip)
- You may need to use [p7zip](https://superuser.com/a/626731) to unzip it

### Look at first 3 lines of raw file

In [48]:
!head -n 3 fma_metadata/tracks.csv

,album,album,album,album,album,album,album,album,album,album,album,album,album,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,set,set,track,track,track,track,track,track,track,track,track,track,track,track,track,track,track,track,track,track,track,track
,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,tags,title,tracks,type,active_year_begin,active_year_end,associated_labels,bio,comments,date_created,favorites,id,latitude,location,longitude,members,name,related_projects,tags,website,wikipedia_page,split,subset,bit_rate,comments,composer,date_created,date_recorded,duration,favorites,genre_top,genres,genres_all,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
track_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


### Read with pandas
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
pd.set_option('display.max_columns', 500)
tracks = pd.read_csv('fma_metadata/tracks.csv', header=[0,1], index_col=0)
tracks.head()

album                                                     \
         comments         date_created        date_released engineer   
track_id                                                               
2               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
3               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
5               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
10              0  2008-11-26 01:45:08  2008-02-06 00:00:00      NaN   
20              0  2008-11-26 01:45:05  2009-01-06 00:00:00      NaN   

                                                                          \
         favorites id                                information listens   
track_id                                                                   
2                4  1                                    <p></p>    6073   
3                4  1                                    <p></p>    6073   
5                4  1                                    <p></p>    6073   
10               4  6                                        NaN   47632   
20               2  4  <p> "spiritual songs" from Nicky Cook</p>    2710   

                                                            \
         producer tags                 title tracks   type   
track_id                                                     
2             NaN   []  AWOL - A Way Of Life      7  Album   
3             NaN   []  AWOL - A Way Of Life      7  Album   
5             NaN   []  AWOL - A Way Of Life      7  Album   
10            NaN   []     Constant Hitmaker      2  Album   
20            NaN   []                 Niris     13  Album   

                       artist                       \
            active_year_begin      active_year_end   
track_id                                             
2         2006-01-01 00:00:00                  NaN   
3         2006-01-01 00:00:00                  NaN   
5         2006-01-01 00:00:00                  NaN   
10                        NaN                  NaN   
20        1990-01-01 00:00:00  2011-01-01 00:00:00   

                                                             \
                                          associated_labels   
track_id                                                      
2                                                       NaN   
3                                                       NaN   
5                                                       NaN   
10        Mexican Summer, Richie Records, Woodsist, Skul...   
20                                                      NaN   

                                                                      \
                                                        bio comments   
track_id                                                               
2         <p>A Way Of Life, A Collective of Hip-Hop from...        0   
3         <p>A Way Of Life, A Collective of Hip-Hop from...        0   
5         <p>A Way Of Life, A Collective of Hip-Hop from...        0   
10        <p><span style="font-family:Verdana, Geneva, A...        3   
20        <p>Songs written by: Nicky Cook</p>\n<p>VOCALS...        2   

                                                                           \
                 date_created favorites id   latitude            location   
track_id                                                                    
2         2008-11-26 01:42:32         9  1  40.058324          New Jersey   
3         2008-11-26 01:42:32         9  1  40.058324          New Jersey   
5         2008-11-26 01:42:32         9  1  40.058324          New Jersey   
10        2008-11-26 01:42:55        74  6        NaN                 NaN   
20        2008-11-26 01:42:52        10  4  51.895927  Colchester England   

                                                                        \
          longitude                                            members   
track_id                                         

### Fit Logistic Regression!

In [50]:
print(tracks.shape)
tracks.isna().sum()


(106574, 52)


album   comments                  0
        date_created           3529
        date_released         36280
        engineer              91279
        favorites                 0
        id                        0
        information           23425
        listens                   0
        producer              88514
        tags                      0
        title                  1025
        tracks                    0
        type                   6508
artist  active_year_begin     83863
        active_year_end      101199
        associated_labels     92303
        bio                   35418
        comments                  0
        date_created            856
        favorites                 0
        id                        0
        latitude              62030
        location              36364
        longitude             62030
        members               59725
        name                      0
        related_projects      93422
        tags                

In [51]:
tracks.columns.tolist()

[('album', 'comments'),
 ('album', 'date_created'),
 ('album', 'date_released'),
 ('album', 'engineer'),
 ('album', 'favorites'),
 ('album', 'id'),
 ('album', 'information'),
 ('album', 'listens'),
 ('album', 'producer'),
 ('album', 'tags'),
 ('album', 'title'),
 ('album', 'tracks'),
 ('album', 'type'),
 ('artist', 'active_year_begin'),
 ('artist', 'active_year_end'),
 ('artist', 'associated_labels'),
 ('artist', 'bio'),
 ('artist', 'comments'),
 ('artist', 'date_created'),
 ('artist', 'favorites'),
 ('artist', 'id'),
 ('artist', 'latitude'),
 ('artist', 'location'),
 ('artist', 'longitude'),
 ('artist', 'members'),
 ('artist', 'name'),
 ('artist', 'related_projects'),
 ('artist', 'tags'),
 ('artist', 'website'),
 ('artist', 'wikipedia_page'),
 ('set', 'split'),
 ('set', 'subset'),
 ('track', 'bit_rate'),
 ('track', 'comments'),
 ('track', 'composer'),
 ('track', 'date_created'),
 ('track', 'date_recorded'),
 ('track', 'duration'),
 ('track', 'favorites'),
 ('track', 'genre_top'),
 ('t

In [52]:
tracks['artist','name'].value_counts()

Kosta T                                   745
Podington Bear                            604
Ergo Phizmiz                              499
Blue Dot Sessions                         416
Lee Rosevere                              371
Mr. & Mrs. Smith                          353
Ars Sonor                                 333
Jahzzar                                   330
James Kibbie                              319
Derek Clegg                               316
Steve Combs                               286
Josh Woodward                             284
Ketsa                                     271
The Agrarians                             260
Jared C. Balogh                           248
Zreen Toyz                                242
OsO El roTo                               231
The Fucked Up Beat                        220
Lezet                                     219
The Polish Ambassador                     214
The Impossebulls                          213
jonas the plugexpert              

In [0]:
tracks2 = tracks.sample(5000, random_state=42)

In [8]:
from sklearn.model_selection import train_test_split

features = [('album','tracks'),('track','interest'),('track','listens'),('track','number')]
target = ('track','genres')

X = tracks2[features]
y = tracks2[target]

X_train,X_test,y_train,y_test = train_test_split(X, y, train_size=.8, test_size=.2, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((4000, 4), (4000,), (1000, 4), (1000,))

In [9]:
log_reg_tracks = LogisticRegression(solver='lbfgs', multi_class='auto')
log_reg_tracks.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [16]:
X_train.head()

album    track               
         tracks interest listens number
track_id                               
133856        9     1685    1488      5
21062         8      576     460      1
42929         2      552     111      1
136897       12     2147    1745      2
13816         6      205     150      6

In [15]:
X_test.head()

album    track               
         tracks interest listens number
track_id                               
23711        38      916     689     13
121504       18     7058    5631      0
148312       27      729     541      5
42490         7      879     213      5
31229        20     5883    5277     13

In [18]:
y_test.head()

track_id
23711             [15, 64, 76, 77, 362]
121504                   [15, 18, 1235]
148312                  [2, 10, 15, 38]
42490                          [32, 38]
31229     [1, 2, 17, 41, 103, 171, 186]
Name: (track, genres), dtype: object

In [10]:
log_reg_tracks.coef_, log_reg_tracks.intercept_

(array([[ 1.57221260e-04,  3.17616239e-04,  1.11108278e-04,
          2.92927794e-05],
        [-6.79538972e-06, -1.39248274e-04,  2.49615600e-05,
         -6.72152299e-07],
        [-3.43771363e-06, -2.88030620e-04, -2.49913243e-04,
         -2.33566237e-06],
        ...,
        [ 2.52497240e-07, -1.64629006e-06, -2.54313209e-04,
         -2.35049595e-06],
        [-2.03706788e-06, -3.58973887e-04, -2.21841415e-04,
          5.33673906e-07],
        [ 3.79259868e-04,  8.72525845e-04, -3.47790728e-04,
          2.05064445e-04]]),
 array([ 6.47938191e-07, -1.81293467e-07, -1.51573608e-07, ...,
         1.11837117e-08, -1.47309112e-07,  1.70469980e-05]))

In [0]:
#Predict X_test
y_pred = log_reg_tracks.predict(X_test)

In [23]:
#Accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.029

In [25]:
#Accuracy
log_reg_tracks.score(X_test, y_test)

0.029

This dataset is bigger than many you've worked with so far, and while it should fit in Colab, it can take awhile to run. That's part of the challenge!

Your tasks:
- Clean up the variable names in the dataframe
- Use logistic regression to fit a model predicting (primary/top) genre
- Inspect, iterate, and improve your model
- Answer the following questions (written, ~paragraph each):
  - What are the best predictors of genre?
  - What information isn't very useful for predicting genre?
  - What surprised you the most about your results?

*Important caveats*:
- This is going to be difficult data to work with - don't let the perfect be the enemy of the good!
- Be creative in cleaning it up - if the best way you know how to do it is download it locally and edit as a spreadsheet, that's OK!
- If the data size becomes problematic, consider sampling/subsetting, or [downcasting numeric datatypes](https://www.dataquest.io/blog/pandas-big-data/).
- You do not need perfect or complete results - just something plausible that runs, and that supports the reasoning in your written answers

If you find that fitting a model to classify *all* genres isn't very good, it's totally OK to limit to the most frequent genres, or perhaps trying to combine or cluster genres as a preprocessing step. Even then, there will be limits to how good a model can be with just this metadata - if you really want to train an effective genre classifier, you'll have to involve the other data (see stretch goals).

This is real data - there is no "one correct answer", so you can take this in a variety of directions. Just make sure to support your findings, and feel free to share them as well! This is meant to be practice for dealing with other "messy" data, a common task in data science.

## Resources and stretch goals

- Check out the other .csv files from the FMA dataset, and see if you can join them or otherwise fit interesting models with them
- [Logistic regression from scratch in numpy](https://blog.goodaudience.com/logistic-regression-from-scratch-in-numpy-5841c09e425f) - if you want to dig in a bit more to both the code and math (also takes a gradient descent approach, introducing the logistic loss function)
- Create a visualization to show predictions of your model - ideally show a confidence interval based on error!
- Check out and compare classification models from scikit-learn, such as [SVM](https://scikit-learn.org/stable/modules/svm.html#classification), [decision trees](https://scikit-learn.org/stable/modules/tree.html#classification), and [naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html). The underlying math will vary significantly, but the API (how you write the code) and interpretation will actually be fairly similar.
- Sign up for [Kaggle](https://kaggle.com), and find a competition to try logistic regression with
- (Not logistic regression related) If you enjoyed the assignment, you may want to read up on [music informatics](https://en.wikipedia.org/wiki/Music_informatics), which is how those audio features were actually calculated. The FMA includes the actual raw audio, so (while this is more of a longterm project than a stretch goal, and won't fit in Colab) if you'd like you can check those out and see what sort of deeper analysis you can do.